In [2]:
#Imports correspondientes.
import urllib.request
import json
import difflib
from bottle import route, run, template

In [3]:
#Parametros de configuracion

#AÑADIR AQUÍ LAS CREDENCIALES PARA SU FUNCIONAMIENTO.

#EDAMAM
edamam_endpoint=""
app_id=""
app_key=""

#DANDELION
dandelion_endpoint=""
dandelion_key= ""
 



In [4]:
#Diccionario que contendrá las distintas unidades de medida que contempla el sistema
measures_dict={}

measures_dict["ml"] = None
measures_dict["l"]= None
measures_dict["gr"]= None
measures_dict["mg"]= None
measures_dict["kg"]= None
measures_dict["miligrams"]= None
measures_dict["grams"]= None
measures_dict["kilograms"]= None
measures_dict["mililiters"]= None
measures_dict["liters"]= None

In [5]:

#FUNCIONALIDADES DEL SERVICIO REST

#Dado un plato, devuelve su receta.
def get_recipe(plate):
    plate=parse_blank_spaces(plate)
    contents = get_edamam_recipe(plate)
    
    #contents es de tipo bytes. Es necesario pasarlo a json.
    contents = parse_bytes_to_JSON(contents)
    best_hit=get_best_hit(plate,contents)["recipe"]["ingredientLines"] #Devuelve una lista con los ingredientes sin procesar
    return parse_recipe(best_hit)




In [6]:
#Métodos auxiliares


'''
    Dado un conjunto de recetas encontradas, devuelve la que encaja más con la entrada introducida por el usuario.
    La receta que encaja más con la del usuario es aquella que tiene mayor semejanza a nivel de cadenas (caracteres).

'''
def get_best_hit(plate,contentJSON):
    #Para cada posible match, buscamos el que tiene mayor similaridad.
    #La similaridad se busca mediante el encaje de cadenas. BUSCAR MEJORA.
    
    max=0    
    for x in contentJSON["hits"] :
        similarity=get_label_similarity(plate,x["recipe"]["label"])
        if(similarity>max):
            max=similarity
            best_hit=x
            
    return best_hit


'''
    Dada una receta, se parsea tal y como se explica abajo.

    Entrada: 
        Lista de ingredientes sin parsear. Un ejemplo de ingrediente podría ser:
        
            "3 medium-sized Yukon Gold potatoes (about 1 1/2 pounds), peeled and quartered lengthwise"
    
    Salida:
        Lista de ingredientes parseados del siguiente modo:
            Para cada ingrediente se crea un diccionario con 3 elementos:
                1- Número de unidades de ese ingrediente (puede no indicar las unidades, en ese caso vale None).
                2- Unidades (mg, gr, etc) de ese ingrediente.
                3- Nombre del ingrediente
'''
def parse_recipe(input):
    result=[]
    
    #Para cada ingrediente.
    for x in input:
        
        d={"num":None, "units":None, "name":None }
        
        if((x.split()[0]).isdigit()):
            d["num"]= int(x.split()[0])
        if(is_measure_unity(x.split()[1])):
            d["units"]= x.split()[1]
            
        #PONER EL VALOR QUE CORRESPONDE.
        d["name"]=x.split()[3]
        
        
        result.append(d)
            
    return result


'''
    Dada una entrada (supuestamente el nombre de un plato), parsea su contenido sustituyendo los espacios
    en blanco por "%20".
    
    Esto se utiliza cuando el plato está compuesto por múltiples palabras (por ejemplo, tortilla de patata).
    En el caso de que el nombre del plato esté compuesto por una sola palabra (por ejemplo, lentejas), este método
    no hará nada.
    
    Para realizar una petición get, es necesario convertir:
        tortilla de patatas ==>  tortilla%20de%20patatas
        
    el resultado es un string parseado tal y como se indica arriba.
    
'''
def parse_blank_spaces(input):
    #Si no tenemos una sola palabra, parsea. En caso contrario, no hace nada.
    return input.replace(" ", "%20") 

'''
    Dado un objeto bytes, obtiene el json asociado.
    
    Se utiliza para formatear el resultado de las llamadas a APIs que devuelven un json.
    
    input: objeto tipo bytes a formatear.
    
    el resultado es un objeto json (diccionario)
    
'''    
def parse_bytes_to_JSON(input):
    decoded = input.decode('utf8') #Decodificamos usando utf-8. El resultado es un string con forma de json.
    return json.loads(decoded);  #Creamos el json a partir del string    


'''
    Obtiene la similaridad entre dos cadenas.
    
    Se usa para comparar las etiquetas de la receta que se solicita y las que encuentra el sistema.
    
    first: primera etiqueta a comparar.
    second: etiqueta que se comparara a first.

    el resultado es un porcentaje.

'''
def get_label_similarity(first, second):
    #Comparará las dos entradas en minuscula.
    seq = difflib.SequenceMatcher(None,first.lower(),second.lower())
    return seq.ratio()*100

def is_measure_unity(word):
    return (word in measures_dict)

def get_edamam_recipe(name):
    return  urllib.request.urlopen(edamam_endpoint+ app_id + "&app_key="+app_key+"&q=+"+name).read()

def get_entities_dandelion(text):
    return urllib.request.urlopen(generaURL(text)).read()


def genera_dandelion_URL(input):
        textURL = "text="
        topEntitiesURL= "top_entities=10"
        includeURL = "include=types%2Ccategories"
    
        token = "token="+dandelion_key;
        splitInput=input.split()
        
        for x in input.split():
            textURL = textURL + x + "+"
            
        textURL = textURL + splitInput[len(splitInput)-1]
        
        return dandelion_endpoint + textURL +"&"+topEntitiesURL+"&"+includeURL+ "&"+token


In [ ]:
#ESQUELETO DEL SERVICIO REST

@route('/recipe/<plate>')
def index(plate):
    #return get_recipe(plate)
    return str(get_recipe(plate))

    
run(host='localhost', port=8080)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



{'num': 2, 'units': None, 'name': 'Bread'}
{'num': 2, 'units': None, 'name': 'cloves'}
{'num': 1, 'units': None, 'name': 'Cumin'}
{'num': None, 'units': None, 'name': 'taste'}
{'num': 3, 'units': None, 'name': '(ripest,'}
{'num': None, 'units': None, 'name': 'cucumber,'}
{'num': 1, 'units': None, 'name': 'bell'}
{'num': None, 'units': None, 'name': 'red'}
{'num': None, 'units': None, 'name': 'Virgin'}
{'num': None, 'units': None, 'name': 'water(chilled)'}
{'num': 3, 'units': None, 'name': 'Vinegar'}


127.0.0.1 - - [12/Mar/2019 17:17:49] "GET /recipe/gazpacho HTTP/1.1" 200 516


In [ ]:
print(get_label_similarity("MUNDO","Mundo"))

In [ ]:
def generaURL(input):
    
        textURL = "text="
        topEntitiesURL= "top_entities=10"
        includeURL = "include=types%2Ccategories"
        
        token = "token="+dandelion_key;
        splitInput=input.split()
        
        for x in input.split():
            textURL = textURL + x + "+"
            
        textURL = textURL + splitInput[len(splitInput)-1]
        
        return dandelion_endpoint + textURL +"&"+topEntitiesURL+"&"+includeURL+ "&"+token
            

contents = urllib.request.urlopen(generaURL("Me gusta la tortilla de patatas y las albondigas")).read()

print(contents)
        